# Explorando las formaciones frente a errores en la percepción
## A partir de "Maneuvering and robustness issues in undirecter displacemente-consensus-based formation control" de Héctor García de Marina
En el artículo se modelan los errores en la percepción como un error de escalado y una matriz de rotación que representan un alineado incorrecto en la medida de las posiciones.
$$ (p_i-p_j)_{measurement}=aR(p_i-p_j)$$
$$ a \in \mathcal{R}, R \in \mathcal{SO}(m)$$


## Reproduciendo los resultados del artículo

### Carga de módulos necesarios

In [ ]:
import RobotModels as rob
import numpy as np
import matplotlib.pyplot as plt
import graph_utils as graf
import time
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from matplotlib.animation import PillowWriter
import robot_plot_utils as rpu

### Formación inicial con 9 agentes
Generación de las posiciones iniciales y el grafo.
Generación de los factores de escala y matriz de rotación responsable de los errores de alineación

In [5]:
#Posiciones iniciales
pi=np.array([[-10,-10],[-10,0],[-10 ,10],[0,10],[0,0],[0,-10],[10,-10],[10,0],[10,10]])
p=np.zeros([2,9])
numnodos,m=np.shape(pi)
#Formacion
zstar=np.array([-5,-5])
#Grafo
Z=np.array([[1,2],[2,3],[3,4],[4,5],[5,6],[5,8],[8,7],[8,9]])
numarcos,m=np.shape(Z)
# Scale factors
var=0.5
a = 1+2*var*np.random.rand(1,9)-var*np.ones([1,9])
#a=np.ones([1,9])
print(a)
#Misalignments in radians
rm=10*np.pi/180
#rm=0
r=np.zeros([1,9])
r=2*rm*np.random.rand(1,9)-rm*np.ones([1,9])
print(r)
n,m= np.shape(pi)
for i in range(n):
    plt.plot(pi[i,0],pi[i,1],'o')

rpu.dibuja_grafo_2D(pi,Z)

[[1.12496026 1.34970896 0.68734953 0.98041717 1.21226484 1.22688971
  1.45526798 0.62776616 0.74804963]]
[[-0.04081941 -0.15724339 -0.09701314 -0.02491326  0.08158734  0.05436008
  -0.07359961 -0.07423069 -0.15465217]]


### Construcción de las matrices
Construimos la matriz Dx $D_x=D_a \otimes I_m \dot D_R$

In [6]:
m,n=np.shape(a)
Da=np.diag(a[0,:])
#print(Da)
j=0
k=0
DR=np.zeros([n*2,2*n])
for i in range(0,2*n,2):
    DR[j,i]=np.cos(r[0,k])
    DR[j,i+1]=-np.sin(r[0,k])
    DR[j+1,i]=np.sin(r[0,k])
    DR[j+1,i+1]=np.cos(r[0,k])
    j+=2
    k+=1
Im=np.eye(2)
Da_bar=np.kron(Da,Im)
i=n*4
#print(np.shape(DR))
#print(Dr)
Dx=np.dot(Da_bar,DR)
#print(np.shape(Dx))
pesos=np.ones([numnodos,numnodos])
L=graf.matriz_laplaciana(Z,pesos,numnodos,numarcos)
print("L: ")
print(L)
Lbar=np.kron(L,Im)
M=-np.dot(Dx,Lbar)

L: 
[[ 1. -1.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  2. -1.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  3. -1.  0. -1.  0.]
 [ 0.  0.  0.  0. -1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -1.  0.]
 [ 0.  0.  0.  0. -1.  0. -1.  3. -1.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  1.]]


### Ejecución del ejemplo

In [7]:
p=np.zeros([2*n,1])
#print(np.shape(p))
rm=20
pstar=graf.stack(pi)
t=0
tf=100
dt=0.01
nptos=(int)((tf-t)/dt)
print(nptos)
fig2 =plt.figure()
p=2*rm*np.random.rand(2*n)-rm*np.ones(2*n)
posiciones=np.zeros([2*n , nptos])
k=0
Lps=np.dot(Lbar,pstar)
#print(Lps)
#Revisar: Los 5 primeros salen bien pero el resto no
for k in range(nptos):
    pdot=np.dot(M,p)+Lps
    for i in range(0,2*n,2):
        xdot,ydot,thetadot=rob.integrator_point_model(pdot[i],pdot[i+1])
        p[i]+=dt*xdot
        p[i+1]+=dt*ydot
    posiciones[:,k]=p[:]
    t+=dt
print(posiciones[:,k])
colors_f=['bo','go','ro','co','yo','mo','ko','bo','go']
colors_i=['bx','gx','rx','cx','yx','mx','kx','bx','gx']
colors_l=['b','g','r','c','y','m','k','b:','g:']
Z=np.array([[1,2],[2,3],[3,4],[4,5],[5,6],[5,8],[8,7],[8,9]])
j=0
for i in range(n):
    plt.plot(posiciones[j,0],posiciones[j+1,0],colors_i[i])
    plt.plot(posiciones[j,nptos-1],posiciones[j+1,nptos-1],colors_f[i])
    plt.plot(posiciones[j,:],posiciones[j+1,:],colors_l[i])
    j=j+2
#rpu.dibuja_grafo_2D(pi,Z)
pfs=np.zeros([2*n,1])
pfs[:,0]=posiciones[:,nptos-1]
pinis=np.zeros([2*n,1])
pinis[:,0]=posiciones[:,0]
#print(pfs)
pf=graf.unstack(pfs,2)
pini=graf.unstack(pinis,2)
#print(pf)
rpu.dibuja_grafo_2D(pf,Z)
rpu.dibuja_grafo_2D(pini,Z)
plt.show()

10000
[-12.33665458 136.40991739 -12.66134903 146.69179019 -13.0900915
 158.13638704   2.30408258 158.80128352   7.8249785  150.6211132
   7.22538434 141.20783046  22.8721008  144.53150621  22.36074281
 152.46716737  20.48386646 163.57733046]


### Bias
Voy a sustituir el parámetro de escala y la matriz de rotación por un bias 
$$(p_i-p_j)_{measurement}=(p_i-p_j-b_{ij})$$
Los bias se pueden agrupar en una matriz que no tiene por qué ser simétrica
$$B \in \mathcal{R}^{nxn}$$
siendo n el número de agentes
1. Voy a generar una matriz aleatoria $B_1 \in \mathcal{R}^{nxn}$ con elementos $b_{ij}=0$
2.  Asigno un valor aleatorio entre $[0,1]$ a los elementos conectados $ b_{ij}\neq 0$ si $\exists \hspace{1cm} Z(p)=(i,j) \hspace{1cm} \text{or} \hspace{1cm} Z(p)=(j,i)$

In [13]:
#Z=np.array([[1,2],[2,3],[3,4],[4,5],[5,6],[5,8],[8,7],[8,9]])
b=np.zeros([numnodos,numnodos])
for k in range(numarcos):
    i=Z[k,0]
    j=Z[k,1]
    b[i-1,j-1]=np.random.rand(1,1)
    b[j-1,i-1]=np.random.rand(1,1)
print(b)
    

[[0.         0.14385489 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.36515934 0.         0.89632826 0.         0.         0.
  0.         0.         0.        ]
 [0.         0.50480355 0.         0.01189135 0.         0.
  0.         0.         0.        ]
 [0.         0.         0.30962392 0.         0.64460114 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.32244715 0.         0.69544283
  0.         0.88188579 0.        ]
 [0.         0.         0.         0.         0.95929446 0.
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.11575197 0.        ]
 [0.         0.         0.         0.         0.23227841 0.
  0.02416318 0.         0.93320209]
 [0.         0.         0.         0.         0.         0.
  0.         0.76677428 0.        ]]


/tmp/ipykernel_15764/3840394568.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b[i-1,j-1]=np.random.rand(1,1)
/tmp/ipykernel_15764/3840394568.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  b[j-1,i-1]=np.random.rand(1,1)


Pruebo a ejecutar a ver el efecto

In [ ]:
p=np.zeros([2*n,1])
#print(np.shape(p))
rm=20
pstar=graf.stack(pi)
t=0
tf=100
dt=0.01
nptos=(int)((tf-t)/dt)
print(nptos)
fig2 =plt.figure()
p=2*rm*np.random.rand(2*n)-rm*np.ones(2*n)
posiciones=np.zeros([2*n , nptos])
k=0
Lps=np.dot(Lbar,pstar)
#print(Lps)
#Revisar: Los 5 primeros salen bien pero el resto no
for k in range(nptos):
    pdot=np.dot(M,p)+Lps
    for i in range(0,2*n,2):
        xdot,ydot,thetadot=rob.integrator_point_model(pdot[i],pdot[i+1])
        p[i]+=dt*xdot
        p[i+1]+=dt*ydot
    posiciones[:,k]=p[:]
    t+=dt
print(posiciones[:,k])
colors_f=['bo','go','ro','co','yo','mo','ko','bo','go']
colors_i=['bx','gx','rx','cx','yx','mx','kx','bx','gx']
colors_l=['b','g','r','c','y','m','k','b:','g:']
Z=np.array([[1,2],[2,3],[3,4],[4,5],[5,6],[5,8],[8,7],[8,9]])
j=0
for i in range(n):
    plt.plot(posiciones[j,0],posiciones[j+1,0],colors_i[i])
    plt.plot(posiciones[j,nptos-1],posiciones[j+1,nptos-1],colors_f[i])
    plt.plot(posiciones[j,:],posiciones[j+1,:],colors_l[i])
    j=j+2
#rpu.dibuja_grafo_2D(pi,Z)
pfs=np.zeros([2*n,1])
pfs[:,0]=posiciones[:,nptos-1]
pinis=np.zeros([2*n,1])
pinis[:,0]=posiciones[:,0]
#print(pfs)
pf=graf.unstack(pfs,2)
pini=graf.unstack(pinis,2)
#print(pf)
rpu.dibuja_grafo_2D(pf,Z)
rpu.dibuja_grafo_2D(pini,Z)
plt.show()